# Apply
The first intermediate functionality from Pandas that we are going to explore is ```.apply()``` which will allow us to apply a function across each column of a DataFrame or across each row of a DataFrame. To begin we are going to pull down all of the prices for the DOW 30 stocks. The 30 stocks are provided below along with the standard imports we will need for this section. Execute the code in the cell below. Using ```.apply()``` is often a way to speed up the calculations of Pandas.

In [ ]:
# provided code
import pandas as pd
import pandas_datareader.data as web
import datetime as dt

# start date
start = dt.datetime(2017, 1, 1)
# end date
end = dt.datetime(2018, 12, 31)

# dow 30 components
# https://www.cnbc.com/dow-30/
dow_stocks = ['AXP', 'AAPL', 'BA', 'CAT', 
              'CSCO', 'CVX', 'DWDP',
              'XOM', 'GS', 'HD', 'IBM',
              'INTC', 'JNJ', 'KO', 'JPM',
              'MCD', 'MMM', 'MRK', 'MSFT',
              'NKE', 'PFE', 'PG', 'TRV',
              'UNH', 'UTX', 'VZ', 'V',
              'WBA', 'WMT', 'DIS']

## Exercise & Step 1) Get the data
Write a function which uses the ```web.DataReader()``` that we used before to get individual stock data. After you return the DataFrame add a new column called ```ticker``` which is the ticker for the data being retrieved. Your function should be called ```get_data``` and should take the following arguments; ```ticker```,```start_dt```, and ```end_dt```. The function should return the compiled DataFrame. I've taken the liberty of giving you the start of the function below;
```python
def get_data(ticker, start_dt, end_dt):
```

In [ ]:
# your code goes here

## Exercise & Step 2) Loop it
Now write a function which executes a For Loop for every element of the list provided to it. In the For Loop we are going to return the relevant stock price data from the function we wrote previously. We will store each DataFrame in a dictionary where the key is the ```ticker``` and the value is the DataFrame. This function should be called ```get_all_ticker_price_data```. The function should return the dictionary at the end. I have provided the start of the function for you once again;
```python
def get_all_ticker_price_data(ticker_list, start_dt, end_dt):
```

In [ ]:
# your code goes here

## Exercise & Step 3) Execute the code
Now in the cell below execute the functions you have written with the provided ```start``` and ```end``` from the provided code cell at the top of this. The end result should be a dictionary of DataFrames. Your dictionary should be called ```ticker_data```.

In [ ]:
# your code goes here

Now with all the data we need to obtain a DataFrame of just the ```close``` of each DataFrame. We will use the provided code below.

In [ ]:
# provided code

# create an empty list to store each Series
ticker_close_prices = list()
# iterate through our dictionary
for ticker, ticker_df in ticker_data.items():
    # get the close series
    close_prices = ticker_df['close']
    # change the name of the series
    close_prices.name = ticker
    # store the close_prices in the 
    # ticker_close_prices list
    ticker_close_prices.append(close_prices)
    
# create a DataFrame of just the closing prices.
close_price_df = pd.concat(ticker_close_prices, axis=1)
close_price_df.head()

## Apply - Columns
Now lets use our ```close_price_df``` and the ```apply``` method and obtain the 20 day simple moving average to each column. First we will need to write a function to compute the simple moving average.

#### Exercise
Write a function called ```compute_sma``` which takes the argument ```price_series``` and ```window_len``` to compute the moving average of prices. Inside the function you will make use of the ```.rolling(window_len)``` method and chain that method with ```.mean()``` to compute a rolling mean. We will return the rolling mean.

In [ ]:
# your code goes here

Now execute the code below to get a DataFrame of the simple moving averages

In [ ]:
# provided code
sma_df = close_price_df.apply(lambda x: compute_sma(x, 20), axis=0)
sma_df.head()

By setting ```axis=0``` we are telling Pandas that we want to apply our function across each column. We could have accomplished this another way as well if we modified our ```compute_sma``` function. Remember that the focus should always be to write **GENERALIZED** functions so that they can be re-used. This is **NOT** an example of that, but I think it teaches some valuable lessons;
```python
def compute_sma(price_series):
    sma = price_series.rolling(20).mean()
    return sma

sma_df = close_price_df.apply(compute_sma, axis=0)
```
Notice that in this example above we did not have to use a ```Lambda``` function and did not explicitly tell Pandas to pass each column to ```compute_sma``` by actually calling the method (meaning we didnt use ```compute_sma(X)``` we just put ```compute_sma```). These would give the same resulting DataFrame as the provided code except that our ```compute_sma``` function only works for a 20 period simple moving average. This is bad practice but technically still works.
<br><br>
## Apply - Rows
To apply a function to just the rows we need to explicitly set ```axis=1``` since the default value is ```axis=0```.
<br><br>
#### Exercise
Now we are going to write some functionality where each day we will check if the price is greater than the simple moving average computed. If that is True then we want to be Long the security for tomorrow (denoted by ```True``` as opposed to ```False```). This will get a little more complicated but don't fret. The first step is to write a function which called ```establish_position``` which will take the following arguments; ```sma_row``` and ```price_row```. This function will then find all the securities on this day where the price is greater than the simple moving average. The function should return a Pandas Series of ```True``` / ```False``` based on whether or not the criteria is met.

In [ ]:
# your code goes here

Now we have the function to compute our positions, execute the code below to run the function and obtain a new object called ```positions_df```

In [ ]:
# provided code
positions_df = sma_df.apply(lambda x: establish_position(x, close_price_df.loc[x.name]), axis=1)

Theres only one problem though, we don't know the SMA value and Price for each day until the day is over. As a result, we would have to trade on all of this information with a 1 day lag. Execute the code below to shift and lag all of the positions by 1 day. We will mark the first day as having no position.

In [ ]:
# provided code
positions_df_lag = positions_df.shift(1)
positions_df_lag.iloc[0,:] = False

I leave it as a separate exercise for the reader to compute what the cumulative return would be for this strategy across the securities being analyzed.